# commons

> Obsidian App configuration

In [ ]:
#| default_exp commons

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from memexplatform_obsidian.settings import ObsidianConfig
from enum import Enum
import urllib
from dataclasses import dataclass
from abc import ABC, abstractmethod
import pathlib
from pathlib import Path
import hashlib

In [ ]:
#| export
config = ObsidianConfig(); config
# (config.PORTAL/'index.qmd').exists()

ObsidianConfig(PREFIX='/obsidian', OBSIDIAN_VAULT=Path('/Users/rahul1.saraf/rahuketu/programming/notesobs'), OBSIDIAN_DB=Path('/Users/rahul1.saraf/rahuketu/programming/gitea/memexplatform_obsidian/data/memexplatform_obsidian.db'), ENABLE_DB_MODE=True)

In [ ]:
#| export
class MountPaths(Enum):
    gradio_url = config.PREFIX + "_gradio"
    open = "/open"
    tag = "/tag"
    embed="/embed"
    obsidian_url = "obsidian://open"
    edit = "/edit"

    def get_mount_name(self):
        return self.value.rsplit("/")[-1]

    def get_app_url(self):
        return f"{config.PREFIX}{self.value}"

    def to(self, **kwargs):
        if kwargs: return self.get_app_url().rstrip("/")+"?"+"&".join([ f"{k}={urllib.parse.quote(str(v))}" for k,v in kwargs.items()])
        else: return self.get_app_url()

    def with_qparams(self, **kwargs):
        if kwargs: return self.value.rstrip("/")+"?"+"&".join([ f"{k}={urllib.parse.quote(str(v))}" for k,v in kwargs.items()])
        else: return self.value

In [ ]:
#| export
class ResponseTypes(Enum):
    file = 'file'
    blob = 'blob'
    html = 'html'
    pygen = 'pygen'

In [ ]:
#| export
@dataclass
class ExtensionTypes:
    VIDEO_EXTS = (".mp4", ".webm", ".ogg", ".mov", ".mkv")
    AUDIO_EXTS = (".mp3", ".wav", ".ogg", ".m4a", ".flac")
    IMAGE_EXTS = (".png", ".jpg", ".jpeg", ".gif", ".svg", ".webp")
    TEXTLIKE_EXTS = (".md", ".qmd", ".canvas", ".base", ".ipynb")

    @classmethod
    def all(cls):
        return cls.VIDEO_EXTS + cls.AUDIO_EXTS + cls.IMAGE_EXTS + cls.TEXTLIKE_EXTS

In [ ]:
ExtensionTypes.all()

('.mp4',
 '.webm',
 '.ogg',
 '.mov',
 '.mkv',
 '.mp3',
 '.wav',
 '.ogg',
 '.m4a',
 '.flac',
 '.png',
 '.jpg',
 '.jpeg',
 '.gif',
 '.svg',
 '.webp',
 '.md',
 '.qmd',
 '.canvas',
 '.base',
 '.ipynb')

In [ ]:
#| export

class Page(ABC):

    def __init__(self, text, fpath=None, created_time=None, modified_time=None, file_size=None):
        self._text = text
        # self._vault= vault if vault else pathlib.Path(".")
        self._fpath = pathlib.Path(fpath) if fpath else None

        self.created_time = created_time
        self.modified_time = modified_time
        self.file_size = file_size

    
    @property
    def file_extension(self):
        ext = self._fpath.suffix
        if ext in ExtensionTypes.all():
            return ext[1:]
        return None

    @classmethod
    @abstractmethod
    def from_file_path(cls, file_path: Path):
        ...

    @property
    def lockey(self):
        return str(self._fpath.relative_to(config.OBSIDIAN_VAULT)) if self._fpath else None

    @property
    def obsidian_url(self):
        return MountPaths.obsidian_url.with_qparams(vault=config.OBSIDIAN_VAULT.name, file=self.lockey)

    @property
    def parent(self):
        return self._fpath.parent.name


    @property
    def app_url(self):
        return MountPaths.open.to(file=self.lockey, title=self.title)

    @property
    @abstractmethod
    def links(self):
        ...

    @property
    @abstractmethod
    def title(self):
        ...

    @property
    @abstractmethod
    def html(self):
        ...

    @property
    @abstractmethod
    def ast(self):
        ...

    @property
    def text(self):
        return self._text

    @property
    def checksum(self):
        return hashlib.md5(self._text.encode('utf-8')).hexdigest()

In [ ]:
MountPaths.gradio_url.get_mount_name()

MountPaths.open.get_app_url()
MountPaths.open.to(vault='test', file='pages/Name of the game')

'/obsidian/open?vault=test&file=pages/Name%20of%20the%20game'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()